In [19]:
# First, we'll import all the required libraries.
import numpy as np
import pandas as pd

In [20]:
from typing import List, Dict

In [21]:
# In this, we are going to use TfidfVectorizer from the Scikit-learn package again.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
#This dataset contains name, artist, and lyrics for 57650 songs in English.
songs = pd.read_csv('Songdataset.csv')

In [23]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [24]:
# Because of the dataset being so big, we are going to resample only 5000 random songs.
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [25]:
# We can notice also the presence of \n in the text, so we are going to remove it.
songs['text'] = songs['text'].str.replace(r'\n','')

<ipython-input-25-560fd330c60b>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n','')


In [26]:
# we have use TF-IDF vectorizer that calculates the TF-IDF score for each song lyric, word-by-word.
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [27]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [28]:
# We now need to calculate the similarity of one lyric to another. We are going to use cosine similarity.
# We want to calculate the cosine similarity of each item with every other item in the dataset. So we just pass the lyrics_matrix as argument.
cosine_similarities = cosine_similarity(lyrics_matrix)

In [29]:
# Once we get the similarities, we'll store in a dictionary the names of the 50 most similar songs for each song in our dataset.
similarities = {}

In [30]:
for i in range(len(cosine_similarities)):
    # we'll sort each element in cosine_similarities and get the indexes of the songs.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    # we'll store in similarities each name of the 50 most similar songs.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [31]:
# we'll define our Content based recommender class.
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} Recommended Songs for "{song}" are:')
        for i in range(rec_items):
            print(f"Song {i+1}:")
            print(f"-> {recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} Similarity score.") 
            print("--------------------------------------------------------------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [32]:
# instantiate class
recommedations = ContentBasedRecommender(similarities)

In [33]:
# we are ready to pick a song from the dataset and make a recommendation.
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [34]:
recommedations.recommend(recommendation)

The 4 Recommended Songs for "Gaston" are:
Song 1:
-> Everybody Have Fun Tonight by Wang Chung with 0.063 Similarity score.
--------------------------------------------------------------------------
Song 2:
-> He's My Guy by Ella Fitzgerald with 0.06 Similarity score.
--------------------------------------------------------------------------
Song 3:
-> I Like It Like That by Van Morrison with 0.057 Similarity score.
--------------------------------------------------------------------------
Song 4:
-> I Guess I Like It Like That by Kylie Minogue with 0.055 Similarity score.
--------------------------------------------------------------------------


In [35]:
# now we again take another random songs and recommend again.
recommendation2 = {
    "song" : songs['song'].iloc[120],
    "number_songs": 4
}

In [36]:
recommedations.recommend(recommendation2)

The 4 Recommended Songs for "Animal" are:
Song 1:
-> Animal by Pearl Jam with 0.594 Similarity score.
--------------------------------------------------------------------------
Song 2:
-> LOVE AND VIOLENCE by Omd with 0.2 Similarity score.
--------------------------------------------------------------------------
Song 3:
-> Sad If I Lost It by Guided By Voices with 0.147 Similarity score.
--------------------------------------------------------------------------
Song 4:
-> Natural High by Lenny Kravitz with 0.147 Similarity score.
--------------------------------------------------------------------------
